In [2]:
import numpy as np
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [5]:
df = pd.read_csv('covid_toy.csv')

In [22]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [9]:
X_train

,age,gender,fever,cough,city
54,60,Female,99.0,Mild,Mumbai
37,55,Male,100.0,Mild,Kolkata
50,19,Male,101.0,Mild,Delhi
99,10,Female,98.0,Strong,Kolkata
83,17,Female,104.0,Mild,Kolkata
...,...,...,...,...,...
88,5,Female,100.0,Mild,Kolkata
95,12,Female,104.0,Mild,Bangalore
65,69,Female,102.0,Mild,Bangalore
94,79,Male,NaN,Strong,Kolkata


## 1. Aam Zindagi

In [10]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [17]:
X_train_fever[:10]

array([[ 99.],
       [100.],
       [101.],
       [ 98.],
       [104.],
       [ 98.],
       [ 98.],
       [102.],
       [ 98.],
       [ 98.]])

In [18]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [19]:
X_train_cough[:10]

array([[0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.]])

In [12]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [20]:
X_train_gender_city[:10]

array([[0., 0., 0., 1.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [13]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [21]:
X_train_age[:10]

array([[60],
       [55],
       [19],
       [10],
       [17],
       [ 5],
       [83],
       [82],
       [24],
       [40]], dtype=int64)

In [14]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [23]:
X_train_transformed[:10]

array([[ 60.,  99.,   0.,   0.,   0.,   1.,   0.],
       [ 55., 100.,   1.,   0.,   1.,   0.,   0.],
       [ 19., 101.,   1.,   1.,   0.,   0.,   0.],
       [ 10.,  98.,   0.,   0.,   1.,   0.,   1.],
       [ 17., 104.,   0.,   0.,   1.,   0.,   0.],
       [  5.,  98.,   0.,   0.,   0.,   1.,   1.],
       [ 83.,  98.,   1.,   1.,   0.,   0.,   0.],
       [ 82., 102.,   0.,   0.,   0.,   0.,   1.],
       [ 24.,  98.,   1.,   0.,   1.,   0.,   0.],
       [ 40.,  98.,   0.,   1.,   0.,   0.,   1.]])

## Mentos Zindagi

In [24]:
from sklearn.compose import ColumnTransformer

In [31]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')
transformer.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first', sparse=False),
                                 ['gender', 'city'])])

In [32]:
transformer.fit_transform(X_train)[:10]

,tnf1__fever,tnf2__cough,tnf3__gender_Male,tnf3__city_Delhi,tnf3__city_Kolkata,tnf3__city_Mumbai,remainder__age
54,99.0,0.0,0.0,0.0,0.0,1.0,60
37,100.0,0.0,1.0,0.0,1.0,0.0,55
50,101.0,0.0,1.0,1.0,0.0,0.0,19
99,98.0,1.0,0.0,0.0,1.0,0.0,10
83,104.0,0.0,0.0,0.0,1.0,0.0,17
98,98.0,1.0,0.0,0.0,0.0,1.0,5
53,98.0,0.0,1.0,1.0,0.0,0.0,83
35,102.0,1.0,0.0,0.0,0.0,0.0,82
82,98.0,0.0,1.0,0.0,1.0,0.0,24
17,98.0,1.0,0.0,1.0,0.0,0.0,40


In [33]:
transformer.transform(X_test)

,tnf1__fever,tnf2__cough,tnf3__gender_Male,tnf3__city_Delhi,tnf3__city_Kolkata,tnf3__city_Mumbai,remainder__age
39,103.000000,0.0,0.0,0.0,1.0,0.0,50
66,104.000000,0.0,1.0,0.0,1.0,0.0,51
87,101.000000,1.0,1.0,0.0,0.0,0.0,47
68,104.000000,1.0,0.0,0.0,1.0,0.0,54
57,99.000000,1.0,0.0,0.0,0.0,0.0,49
38,101.000000,0.0,0.0,1.0,0.0,0.0,49
8,100.000000,1.0,0.0,0.0,0.0,0.0,19
28,104.000000,0.0,1.0,0.0,1.0,0.0,16
21,98.000000,0.0,1.0,0.0,0.0,0.0,73
42,100.000000,0.0,1.0,1.0,0.0,0.0,27
